# Incontro ISLAB 2017-03-23 | Analisi del testo

### Prima di cominciare

- Per prima cosa bisogna considerare cosa definiamo per testo e quali processi di normalizzazione sono necessari.
- Definire la gerarchia/granularità da aggiungere come metadati (es. verso, strofe, poesia)

### Tokenizzazione

- Alcune operazioni di tokenizzazione dipendono dalla lingua del testo: caratteri speciali, punteggiatura
- In alcuni casi bisogna trattare il dominio (hashtag in un tweet invece che in un altro contesto)
- Fenomeni morfologici: trattini per parole composte
- Problemi di encoding

Es. separazione con gli spazi:

- maiuscole e minuscole: "Quel ramo del lago di Como" -> non posso sempre eliminarle
- punteggiatura: non ha senso all'interno di un token

In python esiste `nlt` che contiene diversi **tokenizer**.
Quando uso una regex, devo indicare tutto ciò che voglio catturare.

Una volta ottenuti dei token, possiamo seguire diversi approcci:

- mantenere tutto (maiuscole, minuscole, stopwords etc.) -> costruisco un dizionario enorme (motori di ricerca)
- applicare dei filtri

Esistono molti modi per capire in quale lingua è scritto il testo:

In [2]:
from nltk.corpus import stopwords

def detect_language(text_or_tokens, tokenize=True):
    languages_rations = {}
    lang = 'english'
    if tokenize:
        tokens = tokenizer.wordpunct_tokenize(text_or_tokens)
        words = [x.lower() for x in tokens]
    else:
        words = text_or_tokens
    for language in stopwords.fileids():
        stopwords_set = set([x.decode('utf-8').encode('utf-8') for x in stopwords.words(language)])
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)
        languages_ratios[language] = len(common_elements)
    choice = sorted(languages_ratios.itmes(), key=lambda x: -x[0])
    # mancano poche righe

Due modi per definire le stopwords:

- liste predefinite (es. nltk)
- lista custom (lista di dominio, oppure possiamo usare l'idf per tagliare le parole meno significative) -> idf basso

Come tagliamo questa distribuzione?

- definiamo una soglia statica a priori: non tiene conto della distribuzione
- posso decidere di prendere un certo percentile (max 5%) -> usare un corpus generico rappresentativo della lingua

Le parole hanno una morfologia che cambia anche quando non cambia il significato:

- plurale/singolare
- coniugazione dei verbi

Vantaggi:

- riduciamo la dimensione del dizionario
- si individua con maggior efficacia la semantica

Svantaggi:
- si perde la visiblità degli aspetti sintattici

Criticità:

- La negazione pone molti problemi
- Ordine delle parole -> posting list dell'offset

#### Stemming

Normalizzazione puramente sintattica. Ricondurre le parole a una forma unica codifica che non è una radice della parola. Es. le parole `query` e `querying` diventano `queri`.

Tutti questi meccanismi sono language dependent. Si eliminano suffissi comuni e si riconducono a una parola normalizzata.

Questa operazione non è reversibile (conviene salvarsi un risultato) e potrebbe riconciliare parole che non hanno lo stesso significato.

In [3]:
from nltk.stem.snowball import SnowballStemmer
tokens = ['play', 'playing', 'is', 'are', 'caresses', 'ponies', 'pony']
stemmer = SnowballStemmer('english')
print [stemmer.stem(x) for x in tokens]

[u'play', u'play', 'is', u'are', u'caress', u'poni', u'poni']


#### Lemmatization

Questa operazione si fa con qualche tipo di thesaurus (es. wordnet). Si cerca di ricondurre alla voce del dizionario.

Ha due problemi:

- non ha senso usare wordnet senza avere il sinset della parola

In [15]:
from nltk.corpus import wordnet as wn
tokens = ['play', 'playing', 'is', 'are', 'caresses', 'ponies', 'pony']
for t in tokens:
    print t
    synsets = wn.synsets(t)
    for s0 in synsets[0:1]:
        print s0.pos()
        print [l.name() for l in s0.lemmas()]
        for h in s0.hypernyms():
            print [l.name() for l in h.lemmas()]

play
n
[u'play', u'drama', u'dramatic_play']
[u'dramatic_composition', u'dramatic_work']
playing
n
[u'playing']
[u'musical_performance']
is
v
[u'be']
are
n
[u'are', u'ar']
[u'area_unit', u'square_measure']
caresses
n
[u'caress']
[u'stroke', u'stroking']
ponies
n
[u'pony']
[u'horse', u'Equus_caballus']
pony
n
[u'pony']
[u'horse', u'Equus_caballus']


#### Phrase Queries

**Biwords** (or phrase) indexes
due o più parole consecutive
es. New York, Don Abbondio

- Possiamo indicizzarle a parte
- Sostituiamo le voci

Un caso generale è quello di co-occorenza: non ci interessa che compaiano consecutivamente, ma che siano compresenti nello stesso testo.
Si definisce una finestra nel testo entro cui valutare la co-occorrenza.

- indicizziamo tutte le coppie di parole
- vediamo le ripezioni di ogni coppia
- frequenza relativa rispetto alla parola (modo corretto di stimare la probabilità)
- vedere quanto spesso compaiono insieme rispetto alla occorrenze totali di uno dei due termini

$p(ab)_{attesa} = p(a) * p(b)$

[mutual information](https://en.wikipedia.org/wiki/Mutual_information) as relevance of a 2-word

$m(x, y) = p(x,y) log(p(x,y)/(p(x)*p(y)))

[https://books.google.com/ngrams](https://books.google.com/ngrams)

#### Permuterm Index

#### Spelling correction

#### Phonetic correction

es. soundex algorithm